In [1]:
# randomNG = 'Philox'
randomNG = 'PCG64DXSM'
# randomNG = 'MT19937'
# randomNG = 'SFC64'
import importlib

pass_abc ='ng_sf00'

In [2]:
import numpy as np
import time
import datetime
import numba
import itertools
from scipy.stats import betabinom
import concurrent.futures
import math
import numpy.random
bitNG = getattr(numpy.random, randomNG)
from numpy.random import Generator, SeedSequence
from nonsense_grammar_FNs_004 import play_sequence

In [3]:

runs = 1000

rein1 = 4
rein2 = 0
punish1 = 0
punish2 = -10
iteration_block_length = 10**4
timesteps = 10**7
rand_nsteps = 10 # how many timesteps do we generate random values for at once


grammar_length = 3
grammar_num_terms = 5
# number of grammar types that we'll test on
num_gtest = 1 #differing from scratach which used 3 here
# number of terms that will be novel for the test
num_ttest = 2
# maximum magnitude that can be transmitted will determine number of buckets for each type of stimuli
maxmagnitude = 10
# getting that beta binomial for the sigweights
inertia = 1
alpha = 1
beta = 1



In [4]:
start = time.perf_counter()

sg = SeedSequence()
rgs = [Generator(bitNG(s)) for s in sg.spawn(runs)]

thrds = 25

final_mp = np.zeros((runs, 4))
print('*********************************************')
print('*********************************************')

inputs = []

with concurrent.futures.ProcessPoolExecutor(max_workers=thrds) as executor:
    future_to_nongram_full = {executor.submit(play_sequence, rein1, rein2, punish1, punish2, iteration_block_length, timesteps, rand_nsteps, grammar_length, grammar_num_terms, num_gtest, num_ttest, maxmagnitude, inertia, alpha, beta, rgs, r): inputs for r in range(runs)}
    for future in concurrent.futures.as_completed(future_to_nongram_full):
        inputs = future_to_nongram_full[future]
        try:
            data = future.result()
        except Exception as exc:
            print(f'generated an exception: y? and {exc}')
        else:
            final_mp[data[4]] = data[0:4]



final_mp = np.asarray(final_mp)
cumsuc_mp = np.asarray(final_mp[:,0])
finalsuc_mp = np.asarray(final_mp[:,1])
testAsuc_mp = np.asarray(final_mp[:,2])
testBsuc_mp = np.asarray(final_mp[:,3])

meancumsuc_mp = np.sum(cumsuc_mp)/runs
meanfinalsuc_mp = np.sum(finalsuc_mp)/runs
meantestAsuc_mp = np.sum(testAsuc_mp)/runs
meantestBsuc_mp = np.sum(testBsuc_mp)/runs

print(f'meancumsuc_mp {meancumsuc_mp}')
print(f'meanfinalsuc_mp {meanfinalsuc_mp}')
print(f'meantestAsuc_mp {meantestAsuc_mp}')
print(f'meantestBsuc_mp {meantestBsuc_mp}')

finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')

*********************************************
*********************************************
meancumsuc_mp 9886765.17
meanfinalsuc_mp 994.776
meantestAsuc_mp 616.094
meantestBsuc_mp 735.415
Finished in 10.433333333333334 minutes


In [19]:
cutoff = 900
final_test_count = 0
for cumsum in testBsuc_mp:
    if cumsum > cutoff:
        final_test_count += 1
print(final_test_count)

335


In [14]:
cutoff = 850
final_test_count = 0
for cumsum in finalsuc_mp:
    if cumsum > cutoff:
        final_test_count += 1
print(final_test_count)

1000


In [13]:
cutoff = 990
final_test_count = 0
for cumsum in finalsuc_mp:
    if cumsum > cutoff:
        final_test_count += 1
print(final_test_count)

911
